<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>

# Hyperparameter Tuning

## *Data Science Unit 4 Sprint 2 Assignment 4*

## Your Mission, should you choose to accept it...

To hyperparameter tune and extract every ounce of accuracy out of this telecom customer churn dataset: <https://drive.google.com/file/d/1dfbAsM9DwA7tYhInyflIpZnYs7VT-0AQ/view> 

## Requirements

- [X] Load the data
- [X] Clean the data if necessary (it will be)
- Create and fit a baseline Keras MLP model to the data.
- Hyperparameter tune (at least) the following parameters:
 - batch_size
 - training epochs
 - optimizer
 - learning rate (if applicable to optimizer)
 - momentum (if applicable to optimizer)
 - activation functions
 - network weight initialization
 - dropout regularization
 - number of neurons in the hidden layer
 
 You must use Grid Search and Cross Validation for your initial pass of the above hyperparameters
 
 Try and get the maximum accuracy possible out of this data! You'll save big telecoms millions! Doesn't that sound great?... wtf, no? lmao


In [38]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import LabelEncoder

In [39]:
df = pd.read_csv("WA_Fn-UseC_-Telco-Customer-Churn.csv")

## CLEANING!

In [40]:
# Check for nulls, no null
df.head(5)

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [41]:
def cleanup(df):
    
    # Make all yes and no binary questions 1 and 0
    
    dfx = df.replace({'Yes':1,'No':0,
               'No internet service':-1,
               'No phone service':-2,
               'DSL':1,'Fiber optic':2,
               'Month-to-month':0,'One year':1,
               'Two year':2,
               'Electronic check':3, 'Credit card (automatic)':2, 
               'Bank transfer (automatic)':1, 'Mailed check':0,
               ' ':0})
    dfx['TotalCharges'] = pd.to_numeric(dfx['TotalCharges'])
    # Establish target
    dfy = dfx['Churn']
    # I wrote dfy = df['Churn'] instead

    # Drop ID and gender
    dfx.drop(['customerID', 'gender', 'Churn'], axis='columns', inplace=True)

    # Turn apropos columns one-hot style
    cat_col = ['MultipleLines','InternetService','OnlineSecurity','OnlineBackup',
               'DeviceProtection','TechSupport','StreamingTV','StreamingMovies',
               'Contract','PaymentMethod']
    dfx = pd.get_dummies(dfx, columns=cat_col, prefix_sep='_', drop_first=True)
    return np.array(dfx), np.array(dfy)

In [42]:
for i in df.columns:
    print(f'{i} -> {df[i].unique()}\n')

customerID -> ['7590-VHVEG' '5575-GNVDE' '3668-QPYBK' ... '4801-JZAZL' '8361-LTMKD'
 '3186-AJIEK']

gender -> ['Female' 'Male']

SeniorCitizen -> [0 1]

Partner -> ['Yes' 'No']

Dependents -> ['No' 'Yes']

tenure -> [ 1 34  2 45  8 22 10 28 62 13 16 58 49 25 69 52 71 21 12 30 47 72 17 27
  5 46 11 70 63 43 15 60 18 66  9  3 31 50 64 56  7 42 35 48 29 65 38 68
 32 55 37 36 41  6  4 33 67 23 57 61 14 20 53 40 59 24 44 19 54 51 26  0
 39]

PhoneService -> ['No' 'Yes']

MultipleLines -> ['No phone service' 'No' 'Yes']

InternetService -> ['DSL' 'Fiber optic' 'No']

OnlineSecurity -> ['No' 'Yes' 'No internet service']

OnlineBackup -> ['Yes' 'No' 'No internet service']

DeviceProtection -> ['No' 'Yes' 'No internet service']

TechSupport -> ['No' 'Yes' 'No internet service']

StreamingTV -> ['No' 'Yes' 'No internet service']

StreamingMovies -> ['No' 'Yes' 'No internet service']

Contract -> ['Month-to-month' 'One year' 'Two year']

PaperlessBilling -> ['Yes' 'No']

PaymentMethod -> ['Electr

In [43]:
X, y = cleanup(df)

In [44]:
y

array([0, 0, 1, ..., 0, 1, 0])

#### One-hot encode all non big boi numbers

### Other random words will be one-hot encoded probz

## 3. Create and fit a baseline Keras MLP model to the data.

In [45]:
t_size = .25
rand_s = 444

### Normalize 

In [46]:
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import MinMaxScaler

enc = OrdinalEncoder()
scl = MinMaxScaler()

X_e = enc.fit_transform(X, y)
X_s = scl.fit_transform(X_e, y)

In [47]:
y

array([0, 0, 1, ..., 0, 1, 0])

### Split

In [48]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X_s, y, test_size=t_size, random_state=rand_s
)

In [49]:
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((5282, 29), (5282,), (1761, 29), (1761,))

In [50]:
type(X_train), type(y_train)

(numpy.ndarray, numpy.ndarray)

In [51]:
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((5282, 29), (5282,), (1761, 29), (1761,))

## Keras time

In [52]:
X_train.shape[1]

29

In [64]:
from sklearn.model_selection import GridSearchCV

from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier

# Hyperparams
inputs = X.shape[1]
epochs = 25
b_size = 250

# Create model
def c_m():
    
    model = Sequential()
    model.add(Dense(96, input_shape=(inputs,), activation='relu'))
    model.add(Dropout(0.05))
    model.add(Dense(48, activation='relu'))
    model.add(Dropout(0.05))
    model.add(Dense(48, activation='relu'))
    model.add(Dropout(0.05))
    model.add(Dense(48, activation='relu'))
    model.add(Dropout(0.05))
    model.add(Dense(48, activation='relu'))
    model.add(Dropout(0.15))
    model.add(Dense(96, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))

    # Compile model
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

model = KerasClassifier(build_fn=c_m, verbose=0)

param_grid = {'batch_size': [94,95,96,97],
              'epochs':[6,7,8,9,10]}
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=1, verbose=0)
grid_result = grid.fit(X_train, y_train)
"""# Fit model
wade = model.fit(X_train, y_train,
          validation_data=(X_test,y_test),
          batch_size=b_size,
          epochs=epochs,
          verbose=False)
""";

In [65]:
# Teacher's
# Report Results
print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print(f"Means: {mean}, Stdev: {stdev} with: {param}") 

Best: 0.8085952248482124 using {'batch_size': 95, 'epochs': 7}
Means: 0.7993184467104067, Stdev: 0.013392423946955076 with: {'batch_size': 94, 'epochs': 6}
Means: 0.7985611451096446, Stdev: 0.02042837327839333 with: {'batch_size': 94, 'epochs': 7}
Means: 0.802158260562121, Stdev: 0.015554160775237422 with: {'batch_size': 94, 'epochs': 8}
Means: 0.7974252145947583, Stdev: 0.010026212776610718 with: {'batch_size': 94, 'epochs': 9}
Means: 0.7998864240873858, Stdev: 0.012789347257580559 with: {'batch_size': 94, 'epochs': 10}
Means: 0.8017796190770816, Stdev: 0.01081389890106057 with: {'batch_size': 95, 'epochs': 6}
Means: 0.8085952248482124, Stdev: 0.014100240345334279 with: {'batch_size': 95, 'epochs': 7}
Means: 0.8061340373716134, Stdev: 0.015007668529193946 with: {'batch_size': 95, 'epochs': 8}
Means: 0.805187424106699, Stdev: 0.015901693000020735 with: {'batch_size': 95, 'epochs': 9}
Means: 0.8021582631688369, Stdev: 0.011386307485871456 with: {'batch_size': 95, 'epochs': 10}
Means: 0.

## Stretch Goals:

- Try to implement Random Search Hyperparameter Tuning on this dataset
- Try to implement Bayesian Optimiation tuning on this dataset using hyperas or hyperopt (if you're brave)
- Practice hyperparameter tuning other datasets that we have looked at. How high can you get MNIST? Above 99%?
- Study for the Sprint Challenge
 - Can you implement both perceptron and MLP models from scratch with forward and backpropagation?
 - Can you implement both perceptron and MLP models in keras and tune their hyperparameters with cross validation?